<div style="color:#fdd017;text-align:center;font-weight:bolder;font-size:1.5em;background-color: black;">

# 1. Fuentes

</div>

<div style="color:#fdd017;">

## 1.1. Dataset

Fuente de los datos: https://www.kaggle.com/datasets/blastchar/telco-customer-churn/data<br>

</div>

<div style="color:#fdd017;">

El único dataset que se utilizará en este proyecto de Machine Learning es este dataset publicado por IBM Cognos Analytics ([original](https://community.ibm.com/community/user/blogs/steven-macko/2019/07/11/telco-customer-churn-1113)), en el que se organiza la información de clientes de una empresa de telecomunicaciones. Cada fila corresponde a un cliente individual y cada columna detalla sus atributos.

Incluye la siguiente información:

* **Churn:** Indica si el cliente ha **caído del servicio** en el último mes. Es decir, si el cliente ha causado baja.
* **Servicios contratados:** Detalles sobre los servicios a los que cada cliente está suscrito, como teléfono, varias líneas, internet, seguridad en línea, almacenamiento en la nube, protección de dispositivos, soporte técnico y servicios de streaming (TV y películas).
* **Información de la cuenta:** Datos como la antigüedad del cliente, tipo de contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales.
* **Información demográfica:** Características del cliente como género, rango de edad, si tienen pareja y si tienen dependientes.

</div>

<div style="color:#fdd017;">

### 1.1.1. Problema de Clasificación

El reto se formula como un problema de **clasificación binaria**: para cada cliente vamos a predecir si abandonará el servicio (“Churn = Yes”) o permanecerá (“Churn = No”). Al disponer ya de un histórico etiquetado, lo normal es enfocarlo desde el **aprendizaje supervisado**, donde entrenaremos distintos modelos sobre ejemplos con su resultado real y evaluaremos su capacidad de generalizar. Métricas como el AUC-ROC, la precisión, el recall o el F1-score nos permitirán comparar objetivamente los candidatos y elegir el más robusto para el negocio.

De manera complementaria, aplicaremos también **técnicas no supervisadas** para descubrir segmentos de clientes con patrones de comportamiento similares, que permitan a Negocio obtener quizás puntos de vista nuevos hacia los datos y puedan diseñar campañas o productos que ayuden a fidelizar clientes.

</div>

In [ ]:
import pandas as pd

<div style="color:#fdd017;">

## 1.2. Entendiendo el Dataset

</div>

<div style="color:#fdd017;">

### 1.2.1. Exploración inicial

</div>

In [5]:
# Lectura de datos y muestra inicial
df = pd.read_csv('..\\data\\raw\\WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4296,4489-SNOJF,Female,0,Yes,Yes,35,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,No,Month-to-month,No,Electronic check,72.25,2568.55,Yes
4506,5329-KRDTM,Male,1,Yes,No,72,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),77.35,5396.25,No
5331,5583-EJXRD,Male,0,Yes,Yes,44,No,No phone service,DSL,Yes,...,Yes,Yes,No,Yes,Two year,Yes,Credit card (automatic),54.05,2375.2,No
6605,7632-MNYOY,Male,1,No,No,66,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,One year,No,Credit card (automatic),110.90,7432.05,Yes
4096,0829-XXPLX,Female,0,No,No,20,Yes,No,Fiber optic,Yes,...,Yes,No,Yes,No,Month-to-month,Yes,Bank transfer (automatic),89.40,1871.15,No


<div style="color:#fdd017;">

- Con esta muestra inicial ya se puede apreciar que tenemos muchas variables que habrá que transformar para que sean más útiles para los modelos (columnas "Yes/No"), otras que no aportan valor ("customerID"), algunas numéricas (p.ej. "MonthlyCarges") y otras categóricas (como "PaymentMethod").

</div>

In [7]:
# información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


<div style="color:#fdd017;">

- El dataset se compone de **7043 registros y 21 columnas**.
- No hay datos nulos.
- Muchas de las características son de tipo "object", cuando se pueden convertir fácilmente a numéricas.

</div>

In [8]:
# Para las variables con pocos valores diferentes, analizamos cuáles son.
# De esta manera podemos ver qué transformación requiere.
for i in df.columns:
    if df[i].nunique() < 10:
        print('Columna:', i)
        print(df[i].unique())

Columna: gender
['Female' 'Male']
Columna: SeniorCitizen
[0 1]
Columna: Partner
['Yes' 'No']
Columna: Dependents
['No' 'Yes']
Columna: PhoneService
['No' 'Yes']
Columna: MultipleLines
['No phone service' 'No' 'Yes']
Columna: InternetService
['DSL' 'Fiber optic' 'No']
Columna: OnlineSecurity
['No' 'Yes' 'No internet service']
Columna: OnlineBackup
['Yes' 'No' 'No internet service']
Columna: DeviceProtection
['No' 'Yes' 'No internet service']
Columna: TechSupport
['No' 'Yes' 'No internet service']
Columna: StreamingTV
['No' 'Yes' 'No internet service']
Columna: StreamingMovies
['No' 'Yes' 'No internet service']
Columna: Contract
['Month-to-month' 'One year' 'Two year']
Columna: PaperlessBilling
['Yes' 'No']
Columna: PaymentMethod
['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Columna: Churn
['No' 'Yes']


<div style="color:#fdd017;">

- Se confirma que la mayoría de las características de tipo "object" se pueden transformar a numéricas.
- Hay además 3 variables categóricas: "InternetService", "Contract" y "PaymentMethod".
- Confirmamos así mismo que la variable a predecir, nuestro Target, tan sólo muestra 2 valores: "Yes/No".

</div>

<div style="color:#fdd017;">

### 1.2.2. Análisis de las variables

#### Target / Objetivo:
- **Churn**: Indica si el cliente de esta empresa de telecomunicaciones causó baja en el último mes (Sí/No).

#### Características / Variables predictoras:
-   **customerID**: Identificador único de cada cliente.
-   **gender**: Indica si el cliente es hombre o mujer.
-   **SeniorCitizen**: Especifica si el cliente es una persona de la tercera edad (1 para sí, 0 para no).
-   **Partner**: Indica si el cliente tiene pareja (Sí/No).
-   **Dependents**: Indica si el cliente tiene personas dependientes a su cargo (Sí/No).
-   **tenure**: El número de meses que el cliente ha permanecido con la empresa.
-   **PhoneService**: Indica si el cliente tiene servicio telefónico (Sí/No).
-   **MultipleLines**: Indica si el cliente tiene múltiples líneas (Sí, No o Sin servicio telefónico).
-   **InternetService**: El proveedor de servicio de internet del cliente (DSL, Fibra óptica o No).
-   **OnlineSecurity**: Indica si el cliente tiene seguridad online (Sí, No o Sin servicio de internet).
-   **OnlineBackup**: Indica si el cliente tiene servicio de backup online (Sí, No o Sin servicio de internet).
-   **DeviceProtection**: Indica si el cliente tiene protección de dispositivo (Sí, No o Sin servicio de internet).
-   **TechSupport**: Indica si el cliente tiene soporte técnico (Sí, No o Sin servicio de internet).
-   **StreamingTV**: Indica si el cliente tiene servicio de TV por streaming (Sí, No o Sin servicio de internet).
-   **StreamingMovies**: Indica si el cliente tiene servicio de películas por streaming (Sí, No o Sin servicio de internet).
-   **Contract**: El tipo de contrato del cliente (Mensual, Un año o Dos años).
-   **PaperlessBilling**: Indica si el cliente tiene facturación sin papel (Sí o No).
-   **PaymentMethod**: El método de pago del cliente (Cheque electrónico, Cheque por correo, Transferencia bancaria (automática) o Tarjeta de crédito (automática)).
-   **MonthlyCharges**: El importe que se le cobra al cliente mensualmente.
-   **TotalCharges**: El importe total cobrado al cliente.

</div>
